In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)
df.head()

,Unnamed: 0,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text,...,!Кибербезопасность_importance,!Кибербезопасность_base_words_count,cleaned_texttopics,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words
0,0.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,0.0,та новая выс,тот новый выс,...,0.0,0.0,тот новый выс,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,1.0,взаимодействия,взаимодействие,...,0.0,0.0,взаимодействие,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,2.0,годовой отчет за 2020 год,годовой отчёт 2020 год,...,0.0,0.0,годовой отчёт 2020 год,NaN,NaN,Отношения с инвесторами,-4170.0,Отчетность и прозрачность,-4784.0,6.0
3,3.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,3.0,содержание,содержание,...,0.0,0.0,содержание,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,4.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.loc[df["company"] == "reports/ПАО «ВЫМПЕЛКОМ» ", ["company"]] = "reports/ПАО «ВЫМПЕЛКОМ»"

In [4]:
not_nan = df[df["max_topic_exp"].notna()]
# число абзацев соотвествующих теме
reports = not_nan.groupby(["company", "max_topic_exp"])["year"].count().reset_index().rename(columns={"year": "count"})
reports.head()

,company,max_topic_exp,count
0,reports/En+ Group,Биоразнообразие,35
1,reports/En+ Group,Вода,72
2,reports/En+ Group,Воздух,9
3,reports/En+ Group,Закупки и антикоррупция,26
4,reports/En+ Group,Климат,142


In [5]:
# число отчетов, в которых встречается топик
count_topics = reports.groupby(["max_topic_exp"])["company"].count().to_dict()

In [6]:
# число отчетов
total_companies = reports["company"].nunique()

In [7]:
# число тем в отчете
topics_per_report = reports.groupby(["company"])["max_topic_exp"].count().to_dict()

In [8]:
tfidf_company = defaultdict(dict)
for idx, (company, report_topics, count) in reports.iterrows():
    tfidf_company[company][report_topics + "_tf"] = count / topics_per_report[company]
    tfidf_company[company][report_topics + "_idf"] = total_companies / count_topics[report_topics]

In [9]:
tfidf_company = pd.DataFrame(tfidf_company).T.fillna(0)
tfidf_company.to_csv("tfidf_company.csv")
tfidf_company.head()

,Биоразнообразие_tf,Биоразнообразие_idf,Вода_tf,Вода_idf,Воздух_tf,Воздух_idf,Закупки и антикоррупция_tf,Закупки и антикоррупция_idf,Климат_tf,Климат_idf,...,Малый и локальный бизнес_tf,Малый и локальный бизнес_idf,Отходы и циклическая экономика_tf,Отходы и циклическая экономика_idf,Этика_tf,Этика_idf,Отчетность и прозрачность_tf,Отчетность и прозрачность_idf,Лидерство_tf,Лидерство_idf
reports/En+ Group,1.250000,1.020408,2.571429,1.041667,0.321429,1.428571,0.928571,1.020408,5.071429,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
reports/GV Gold,0.160000,1.020408,0.880000,1.041667,0.120000,1.428571,0.760000,1.020408,1.000000,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
reports/X5 Retail Group,0.607143,1.020408,0.821429,1.041667,0.035714,1.428571,0.500000,1.020408,2.785714,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
reports/«МЕТАЛЛОИНВЕСТ»,0.875000,1.020408,2.208333,1.041667,0.208333,1.428571,2.583333,1.020408,4.083333,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
reports/АК «АЛРОСА»,1.730769,1.020408,2.346154,1.041667,0.115385,1.428571,1.769231,1.020408,2.576923,1.020408,...,0.038462,10.0,0.038462,1.785714,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df.fillna("безтемы").groupby(["company", "max_topic_exp"])["year"].count().to_csv("test.csv")

In [25]:
df.fillna("безтемы").groupby(["company"])["year"].count().shape

(50,)

In [22]:
df["company"].nunique()

50

In [11]:
tfidf_company.reset_index(inplace=True)
tfidf_company["index"] = tfidf_company["index"].str.split("/").str[1]
tfidf_company = tfidf_company.set_index("index")
tfidf_company.head()

,Биоразнообразие_tf,Биоразнообразие_idf,Вода_tf,Вода_idf,Воздух_tf,Воздух_idf,Закупки и антикоррупция_tf,Закупки и антикоррупция_idf,Климат_tf,Климат_idf,...,Малый и локальный бизнес_tf,Малый и локальный бизнес_idf,Отходы и циклическая экономика_tf,Отходы и циклическая экономика_idf,Этика_tf,Этика_idf,Отчетность и прозрачность_tf,Отчетность и прозрачность_idf,Лидерство_tf,Лидерство_idf
index,,,,,,,,,,,,,,,,,,,,,
En+ Group,1.250000,1.020408,2.571429,1.041667,0.321429,1.428571,0.928571,1.020408,5.071429,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
GV Gold,0.160000,1.020408,0.880000,1.041667,0.120000,1.428571,0.760000,1.020408,1.000000,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
X5 Retail Group,0.607143,1.020408,0.821429,1.041667,0.035714,1.428571,0.500000,1.020408,2.785714,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
«МЕТАЛЛОИНВЕСТ»,0.875000,1.020408,2.208333,1.041667,0.208333,1.428571,2.583333,1.020408,4.083333,1.020408,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
АК «АЛРОСА»,1.730769,1.020408,2.346154,1.041667,0.115385,1.428571,1.769231,1.020408,2.576923,1.020408,...,0.038462,10.0,0.038462,1.785714,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
raex_scores = pd.read_csv("raex_esg_with_rspp.csv", index_col=0)
raex_scores.head()

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,...,Год последней оцененной отчетности,E_transformed,S_transformed,G_transformed,id,компания,сектор,год,тип отчета,ссылка на отчет
0.0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,...,2021.0,2.0,2.0,3.0,714.0,ПАО «НЛМК»,Металлургическая и горнодобывающая,2021.0,ИО,/download/de45f8ddf53aa9ba80614d1c93e296fc/
1.0,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,...,2021.0,1.0,3.0,3.0,887.0,ПАО «Полюс»,Металлургическая и горнодобывающая,2021.0,ОУР,/download/a79b0f48c2f54dc885ddf583ef4c402d/
2.0,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,...,2021.0,4.0,2.0,2.0,1275.0,ПАО «Уралкалий»,"Химическая, нефтехимическая, парфюмерная",2021.0,ОУР,/download/0e88d52331c02d9939be212b2b95e562/
3.0,4,«ЭЛ5-Энерго»,ELFV,Электроэнергетика,A,9,BBB,9,A,2,...,2021.0,4.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4.0,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,...,2021.0,3.0,3.0,2.0,875.0,ПАО «Полиметалл»,Металлургическая и горнодобывающая,2021.0,ИО,/download/dd60bd2b772498057452e9b81e540671/


In [13]:
assert len(tfidf_company.index[~tfidf_company.index.isin(raex_scores["компания"])]) == 0, tfidf_company.index[~tfidf_company.index.isin(raex_scores["компания"])]

AssertionError: Index(['Газпром', 'Юнипро'], dtype='object', name='index')

In [14]:
tfidf_company.index[~tfidf_company.index.isin(raex_scores["компания"])]

Index(['Газпром', 'Юнипро'], dtype='object', name='index')

In [15]:
combined = tfidf_company.join(raex_scores.set_index("компания")[["E_transformed", "S_transformed", "G_transformed"]])

In [16]:
topic_words = pd.read_csv("../topic_words.csv", index_col=0)
topic_words.head()

,topic,weight,type,word
meta,,,,
env,Экология в целом,5.0,0.0,экологичный
env,Экология в целом,5.0,0.0,природоохранный
env,Экология в целом,1.0,0.0,экология
env,Экология в целом,5.0,0.0,воздействие
env,Экология в целом,5.0,0.0,окружающая среда


In [17]:
combined.to_csv("tfidf_company_with_esg.csv")

In [18]:
gr = topic_words.groupby("meta")["topic"].apply(set)

In [20]:
topics = set([col.split("_")[0] for col in combined.columns if col.find("_") != -1])
topics = [topic for topic in topics if len(topic) > 1]

In [21]:
letter = {"env": ["E_transformed"], "social": ["S_transformed"], "gov": ["G_transformed"]}
for meta, col_set in gr.items():
    for col in col_set:
        for c_col in combined.columns:
            if col in c_col:
                letter[meta].append(c_col)

# Fisrt

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


def logreg_(data: pd.DataFrame, cur_letter: str):
    model = LogisticRegression()
    y_col = f"{cur_letter}_transformed"
    X = data.drop([y_col], axis=1)
    y = data[y_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("accuracy_score", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [ ]:
for cur_letter, letter_type in [("E", "env"), ("S", "social"), ("G", "gov")]:
    print(cur_letter, letter_type)
    logreg_(combined[letter[letter_type]], cur_letter)

E env
accuracy_score 0.5
              precision    recall  f1-score   support

         4.0       1.00      0.50      0.67         2
         6.0       1.00      1.00      1.00         1
         7.0       0.00      0.00      0.00         1
         8.0       0.00      0.00      0.00         0

    accuracy                           0.50         4
   macro avg       0.50      0.38      0.42         4
weighted avg       0.75      0.50      0.58         4

S social
accuracy_score 0.25
              precision    recall  f1-score   support

         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00         1
         5.0       0.50      0.50      0.50         2

    accuracy                           0.25         4
   macro avg       0.17      0.17      0.17         4
weighted avg       0.25      0.25      0.25         4

G gov
accuracy_score 0.5
              precision    recall  f1-score   support

         3.0       1.00      0.50      0.67       

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarnin

# Second

In [ ]:
from pymcdm.methods import PROMETHEE_II

KeyboardInterrupt: 

In [ ]:
gr = (
    df.drop(
        columns=[
            "Unnamed: 0",
            "rsspp_index",
            "index",
            "sector",
            "year",
            "report_type",
            "paragraph",
            "original_text",
            "cleaned_text",
            "max_topic_cos",
            "max_score_cos",
            "count_words",
            "second_max_topic_cos",
            "second_max_score_cos",
            "max_topic_exp",
            "max_score_exp",
            "second_max_topic_exp",
            "second_max_score_exp",
        ]
    )
    .groupby("company")
    .mean()
    .dropna(how="all")
)
gr.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность
company,,,,,,,,,,,,,,,,,,,,,
reports/GV Gold,0.007228,0.024504,0.023391,0.011764,0.019744,0.013694,0.006476,0.011152,0.010913,0.0,...,0.024308,0.001417,0.0,0.0,0.020799,0.021285,0.005739,0.012695,0.026362,0.0
reports/«МЕТАЛЛОИНВЕСТ»,0.005318,0.017873,0.010172,0.007300,0.014439,0.015223,0.002570,0.003172,0.016063,0.0,...,0.026322,0.002332,0.0,0.0,0.031231,0.014672,0.004013,0.016205,0.025135,0.0
reports/АО «СУЭК»,0.011790,0.012718,0.009848,0.011270,0.015054,0.016218,0.003856,0.002934,0.011745,0.0,...,0.038851,0.001876,0.0,0.0,0.034406,0.017499,0.001861,0.023446,0.027961,0.0
reports/Газпромбанк,0.005860,0.024217,0.012030,0.009286,0.008939,0.005113,0.003825,0.002933,0.006976,0.0,...,0.032912,0.001359,0.0,0.0,0.026048,0.016484,0.006207,0.005743,0.035527,0.0
reports/Группа ЛСР,0.007281,0.025014,0.007616,0.013441,0.015713,0.011888,0.006987,0.006941,0.008769,0.0,...,0.034395,0.000992,0.0,0.0,0.026231,0.027838,0.002592,0.008336,0.023474,0.0


In [ ]:
topics = topic_words.groupby("meta")["topic"].apply(set)

In [ ]:
gr.reset_index(inplace=True)
gr["company"] = gr["company"].str.split("/").str[1]
gr = gr.set_index("company")
gr.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность
company,,,,,,,,,,,,,,,,,,,,,
GV Gold,0.007228,0.024504,0.023391,0.011764,0.019744,0.013694,0.006476,0.011152,0.010913,0.0,...,0.024308,0.001417,0.0,0.0,0.020799,0.021285,0.005739,0.012695,0.026362,0.0
«МЕТАЛЛОИНВЕСТ»,0.005318,0.017873,0.010172,0.007300,0.014439,0.015223,0.002570,0.003172,0.016063,0.0,...,0.026322,0.002332,0.0,0.0,0.031231,0.014672,0.004013,0.016205,0.025135,0.0
АО «СУЭК»,0.011790,0.012718,0.009848,0.011270,0.015054,0.016218,0.003856,0.002934,0.011745,0.0,...,0.038851,0.001876,0.0,0.0,0.034406,0.017499,0.001861,0.023446,0.027961,0.0
Газпромбанк,0.005860,0.024217,0.012030,0.009286,0.008939,0.005113,0.003825,0.002933,0.006976,0.0,...,0.032912,0.001359,0.0,0.0,0.026048,0.016484,0.006207,0.005743,0.035527,0.0
Группа ЛСР,0.007281,0.025014,0.007616,0.013441,0.015713,0.011888,0.006987,0.006941,0.008769,0.0,...,0.034395,0.000992,0.0,0.0,0.026231,0.027838,0.002592,0.008336,0.023474,0.0


In [ ]:
def second_workflow(data: pd.DataFrame):
    matrix = gr.to_numpy()
    weights = np.array([1 / matrix.shape[0]] * matrix.shape[1])
    types = np.ones(matrix.shape[1], dtype=int)
    body = PROMETHEE_II("usual")
    print([round(preference, 2) for preference in body(matrix, weights, types)])

In [ ]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]
for key, cur_topics in topics.items():
    print(key)
    tmp_topics = []
    for t in cur_topics:
        if t not in bad:
            tmp_topics.append(t)
        else:
            tmp_topics.append("!" + t)
    cur_topics = tmp_topics
    second_workflow(gr[cur_topics])

env
[0.13, 0.14, 0.39, 0.35, 0.16, -0.35, -0.61, 0.04, -0.02, -0.04, 0.41, 0.14, -1.69, 0.29, 0.38, 0.26, 0.49, 0.12, -0.61, 0.04]
gov
[0.13, 0.14, 0.39, 0.35, 0.16, -0.35, -0.61, 0.04, -0.02, -0.04, 0.41, 0.14, -1.69, 0.29, 0.38, 0.26, 0.49, 0.12, -0.61, 0.04]
social
[0.13, 0.14, 0.39, 0.35, 0.16, -0.35, -0.61, 0.04, -0.02, -0.04, 0.41, 0.14, -1.69, 0.29, 0.38, 0.26, 0.49, 0.12, -0.61, 0.04]
